In [1]:
import pandas as pd
import numpy as np
from pyomo.environ import *

In [2]:
distance_matrix = pd.read_csv('../../data/dataset/Distance_Matrix.csv')
biomass_hist = pd.read_csv('../../data/dataset/Biomass_History.csv')

distance_matrix.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2408,2409,2410,2411,2412,2413,2414,2415,2416,2417
0,0,0.0000,11.3769,20.4557,38.1227,45.3810,54.9915,78.6108,118.6750,102.6639,...,683.8771,687.6310,697.3246,669.3962,667.6788,665.5775,662.0291,665.9655,673.2073,681.4235
1,1,11.3769,0.0000,9.0788,28.9141,36.1724,45.7829,69.4022,78.2329,93.4553,...,681.6295,685.3833,695.0769,667.1485,665.4311,663.3298,659.7815,663.7178,670.9596,679.1758
2,2,20.4557,9.0788,0.0000,22.3791,29.6374,39.2478,62.8671,71.6979,86.9203,...,682.2323,685.9861,695.6796,667.7513,666.0339,663.9326,660.3843,664.3206,671.5623,679.7786
3,3,38.1227,28.9141,22.3791,0.0000,11.8343,23.5413,41.8396,50.6703,65.8927,...,681.4226,685.1765,694.8701,666.9417,665.2243,663.1230,659.5746,663.5110,670.7528,678.9690
4,4,45.3810,36.1724,29.6374,11.8343,0.0000,11.7070,24.3986,33.2293,53.9901,...,663.9816,667.7355,677.4291,649.5007,647.7833,645.6820,642.1336,646.0700,653.3118,661.5280


In [3]:
distance_matrix.drop(['Unnamed: 0'], axis=1, inplace=True)
distance_matrix = np.array(distance_matrix)
distance_matrix

array([[  0.    ,  11.3769,  20.4557, ..., 665.9655, 673.2073, 681.4235],
       [ 11.3769,   0.    ,   9.0788, ..., 663.7178, 670.9596, 679.1758],
       [ 20.4557,   9.0788,   0.    , ..., 664.3206, 671.5623, 679.7786],
       ...,
       [663.7748, 661.5271, 662.1298, ...,   0.    ,  14.2423,  22.4586],
       [671.0165, 668.7688, 669.3715, ...,  14.2423,   0.    ,  12.4741],
       [679.2328, 676.9851, 677.5878, ...,  22.4586,  12.4741,   0.    ]])

In [4]:
b_2010 = np.array(biomass_hist['2010'])

In [5]:
sum(b_2010[:10])/2

183.99356841850002

In [6]:
sum(b_2010[:10])

367.98713683700004

In [7]:
# num_locations = 2418
# num_depots_max = 25
# num_refineries_max = 5
# harvest = b_2010 # List of harvest amounts at each location
# #distance_matrix  2418x2418 distance matrix
# cap_depot = 20000
# cap_refinery = 100000

num_locations = 10
num_depots_max = 2
num_refineries_max = 1
harvest = b_2010[:10] # List of harvest amounts at each location
#distance_matrix  2418x2418 distance matrix
cap_depot = 200
cap_refinery = 350

# Create a concrete model
model = ConcreteModel()

# Index sets
model.locations = RangeSet(num_locations)
model.depots = RangeSet(num_locations)  # We assume all locations can be potential depots
# model.refineries = RangeSet(1, num_refineries_max)
model.refineries = RangeSet(1, num_locations) # We assume all locations can be potential refineries



In [9]:
for i in model.locations:
    print(i)

1
2
3
4
5
6
7
8
9
10


In [11]:
i*2

20

In [10]:
# Decision variables
model.depots_used = Var(model.depots, within=Binary)
model.refineries_used = Var(model.refineries, within=Binary)
model.harvest_flow = Var(model.locations, model.depots, within=NonNegativeReals)
model.pellet_flow = Var(model.depots, model.refineries, within=NonNegativeReals)
# model.under_utilization_cost = Var(model.depots, within=NonNegativeReals)
# model.under_utilization_cost_refineries = Var(model.refineries, within=NonNegativeReals)
model.pellet_flow_active = Var(model.depots, model.refineries, within=Binary)  # Binary variable indicating active pellet flow

In [11]:


## Linear

# # Objective function
def objective_rule(model):
    return (
        0.001*sum(distance_matrix[i-1][j-1] * model.harvest_flow[i, j] for i in model.locations for j in model.depots) +
        0.001*sum(distance_matrix[j-1][k-1] * model.pellet_flow[j, k] for j in model.depots for k in model.refineries) +
        sum(cap_depot - model.harvest_flow[i, j] for i in model.locations for j in model.depots) +  # Under-utilization cost for depots
        sum(cap_refinery - model.pellet_flow[j, k] for j in model.depots for k in model.refineries)   # Under-utilization cost for refineries
        # sum(cap_refinery * (1 - model.refineries_used[k]) for k in model.refineries)  # Under-utilization cost for unused refineries -->redundant
    )
model.objective = Objective(rule=objective_rule, sense=minimize)


## Non Linear
# Objective function
# def objective_rule(model):
#     return (
#         0.001*sum(distance_matrix[i-1][j-1]*model.harvest_flow[i, j]*model.depots_used[j] for i in model.locations for j in model.depots) +
#         0.001*sum(distance_matrix[j-1][k-1]*model.pellet_flow[j, k]*model.refineries_used[k]  for j in model.depots for k in model.refineries) +
#         sum(cap_depot - model.harvest_flow[i, j] for i in model.locations for j in model.depots) +  # Under-utilization cost for depots
#         sum(cap_refinery - model.pellet_flow[j, k] for j in model.depots for k in model.refineries)   # Under-utilization cost for refineries
#         # sum(cap_refinery * (1 - model.refineries_used[k]) for k in model.refineries)  # Under-utilization cost for unused refineries -->redundant
#     )
# model.objective = Objective(rule=objective_rule, sense=minimize)


# Constraints
def max_depots_constraint_rule(model):
    return sum(model.depots_used[j] for j in model.depots) <= num_depots_max
model.max_depots_constraint = Constraint(rule=max_depots_constraint_rule)

# Constraint for maximum number of refineries
def max_refineries_constraint_rule(model):
    return sum(model.refineries_used[k] for k in model.refineries) <= num_refineries_max
model.max_refineries_constraint = Constraint(rule=max_refineries_constraint_rule)

def harvest_supply_constraint_rule(model, i):
    return sum(model.harvest_flow[i, j] for j in model.depots) <= harvest[i-1]
model.harvest_supply_constraint = Constraint(model.locations, rule=harvest_supply_constraint_rule)

# Constraint for pellets flow balance
def pellet_flow_balance_rule(model, j):
    return (
        sum(model.pellet_flow[j, k] for k in model.refineries) ==
        sum(model.harvest_flow[i, j] for i in model.locations)
    )
model.pellet_flow_balance_constraint = Constraint(model.depots, rule=pellet_flow_balance_rule)

def flow_binary_relation_rule(model, i, j):
    """
    This constraint ensures that if a depot is used (the binary variable is 1), 
    the corresponding harvest flow can be non-zero, but if a depot is not used (the binary variable is 0), the corresponding harvest flow must be zero.
    """
    return model.harvest_flow[i, j] <= model.depots_used[j] * harvest[i-1]
model.flow_binary_relation_constraint = Constraint(model.locations, model.depots, rule=flow_binary_relation_rule)


def pellet_flow_binary_relation_rule(model,i, j, k):
    return model.pellet_flow[j, k] <= model.refineries_used[k] * sum(model.harvest_flow[i, j] for i in model.locations)
model.pellet_flow_binary_relation_constraint = Constraint(model.locations, model.depots, model.refineries, rule=pellet_flow_binary_relation_rule)


def max_utilization_constraint_rule(model, j):
    return sum(model.harvest_flow[i, j] for i in model.locations) <= cap_depot
model.max_utilization_constraint = Constraint(model.depots, rule=max_utilization_constraint_rule)


# Constraint for maximum utilization of a refinery
def max_utilization_refineries_constraint_rule(model, k):
    return sum(model.pellet_flow[j, k] for j in model.depots) <= cap_refinery
model.max_utilization_refineries_constraint = Constraint(model.refineries, rule=max_utilization_refineries_constraint_rule)


# Solve the problem
print("Solution started...")
solver = SolverFactory('ipopt')
results = solver.solve(model)

# Display results
print("Solver status:", results.solver.status)
print("Objective value:", value(model.objective))

Solution started...
Solver status: ok
Objective value: 54275.75552484466


In [11]:
print(results)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 8482
  Number of variables: 840
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 465.29777574539185
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [12]:
# Display which locations are used as depots
depots_used = [j for j in model.depots if value(model.depots_used[j]) > 0.5]
print("Depot locations:", depots_used)
refineries_used = [k for k in model.refineries if value(model.refineries_used[k]) > 0.5]
print("Refineries locations:", refineries_used)
# # Display the harvest flow from each location to each depot
# for i in model.locations:
#     for j in depots_used:
#         flow = value(model.harvest_flow[i, j])
#         if flow > 0:
#             print(f"Harvest flow from location {i} to depot {j}: {flow}")

Depot locations: [3, 6, 17, 18, 19]
Refineries locations: [6, 18]


In [14]:
biomass_source_index = []
biomass_destination_index = []
biomass_flow = []
for i in model.locations:
    for j in depots_used:
        f = value(model.harvest_flow[i, j])
        if f > 0:
            biomass_source_index.append(i)
            biomass_destination_index.append(j)
            biomass_flow.append(f)

In [16]:
pellet_source_index = []
pellet_destination_index = []
pellet_flow = []
for i in model.locations:
    for j in refineries_used:
        f = value(model.pellet_flow[i, j])
        if f > 0:
            pellet_source_index.append(i)
            pellet_destination_index.append(j)
            pellet_flow.append(f)

In [5]:
assert (all(x >= 0 for x in [1,2,-3])), "Constraint 1 failed"

AssertionError: Constraint 1 failed

In [6]:
1e-03

0.001

In [8]:
import pandas as pd
import numpy as np
from pyomo.environ import *

distance_matrix = pd.read_csv('../data/dataset/Distance_Matrix.csv')
biomass_hist = pd.read_csv('../data/dataset/Biomass_History.csv')

distance_matrix.drop(['Unnamed: 0'], axis=1, inplace=True)
distance_matrix = np.array(distance_matrix)

b_2010 = np.array(biomass_hist['2010'])


FileNotFoundError: [Errno 2] No such file or directory: '../data/dataset/Distance_Matrix.csv'

In [ ]:
def optimize(distance_matrix, harvests,num_locations = 2418,num_depots_max = 25,num_refineries_max = 5,cap_depot = 20000,cap_refinery = 100000):
       
    #TODO: Harvest needs to be list of lists, (for every year)
    # Create a concrete model
    harvest = harvests[0]
    model = ConcreteModel()
    # Index sets
    model.locations = RangeSet(num_locations)
    model.depots = RangeSet(num_locations)  # We assume all locations can be potential depots
    model.refineries = RangeSet(1, num_locations) # We assume all locations can be potential refineries
    
    # Decision variables
    model.depots_used = Var(model.depots, within=Binary)
    model.refineries_used = Var(model.refineries, within=Binary)
    model.harvest_flow = Var(model.locations, model.depots, within=NonNegativeReals) # *Constraint 1
    model.pellet_flow = Var(model.depots, model.refineries, within=NonNegativeReals) # *Constraint 1
    # model.under_utilization_cost = Var(model.depots, within=NonNegativeReals)
    # model.under_utilization_cost_refineries = Var(model.refineries, within=NonNegativeReals)
    model.pellet_flow_active = Var(model.depots, model.refineries, within=Binary)  # Binary variable indicating active pellet flow

    ## Linear

    # # Objective function
    def objective_rule(model):
        return (
            0.001*sum(distance_matrix[i-1][j-1] * model.harvest_flow[i, j] for i in model.locations for j in model.depots) +
            0.001*sum(distance_matrix[j-1][k-1] * model.pellet_flow[j, k] for j in model.depots for k in model.refineries) +
            sum(cap_depot - model.harvest_flow[i, j] for i in model.locations for j in model.depots) +  # Under-utilization cost for depots
            sum(cap_refinery - model.pellet_flow[j, k] for j in model.depots for k in model.refineries)   # Under-utilization cost for refineries
            # sum(cap_refinery * (1 - model.refineries_used[k]) for k in model.refineries)  # Under-utilization cost for unused refineries -->redundant
        )
    model.objective = Objective(rule=objective_rule, sense=minimize)


    ## Non Linear
    # Objective function
    # def objective_rule(model):
    #     return (
    #         0.001*sum(distance_matrix[i-1][j-1]*model.harvest_flow[i, j]*model.depots_used[j] for i in model.locations for j in model.depots) +
    #         0.001*sum(distance_matrix[j-1][k-1]*model.pellet_flow[j, k]*model.refineries_used[k]  for j in model.depots for k in model.refineries) +
    #         sum(cap_depot - model.harvest_flow[i, j] for i in model.locations for j in model.depots) +  # Under-utilization cost for depots
    #         sum(cap_refinery - model.pellet_flow[j, k] for j in model.depots for k in model.refineries)   # Under-utilization cost for refineries
    #         # sum(cap_refinery * (1 - model.refineries_used[k]) for k in model.refineries)  # Under-utilization cost for unused refineries -->redundant
    #     )
    # model.objective = Objective(rule=objective_rule, sense=minimize)


    # Constraints

    # * Constraint 2
    def harvest_supply_constraint_rule(model, i):
        return sum(model.harvest_flow[i, j] for j in model.depots) <= harvest[i-1]
    model.harvest_supply_constraint = Constraint(model.locations, rule=harvest_supply_constraint_rule)

    # * Constraint 3 
    # for maximum utilization of a Depot
    def max_utilization_constraint_rule(model, j):
        return sum(model.harvest_flow[i, j] for i in model.locations) <= cap_depot
    model.max_utilization_constraint = Constraint(model.depots, rule=max_utilization_constraint_rule)

    # * Constraint 4 
    # for maximum utilization of a refinery
    def max_utilization_refineries_constraint_rule(model, k):
        return sum(model.pellet_flow[j, k] for j in model.depots) <= cap_refinery
    model.max_utilization_refineries_constraint = Constraint(model.refineries, rule=max_utilization_refineries_constraint_rule)

    # *Constraint 5
    def max_depots_constraint_rule(model):
        return sum(model.depots_used[j] for j in model.depots) <= num_depots_max
    model.max_depots_constraint = Constraint(rule=max_depots_constraint_rule)

    # * Constraint 6
    # Constraint for maximum number of refineries
    def max_refineries_constraint_rule(model):
        return sum(model.refineries_used[k] for k in model.refineries) <= num_refineries_max
    model.max_refineries_constraint = Constraint(rule=max_refineries_constraint_rule)

    # * Constraint 7
    # At least 80% of the total forecasted biomass must be processed by refineries each year
    def min_80_constraint_rule(model):
        return sum(model.pellet_flow[j, k] for j in model.depots for k in model.refineries) >= 0.8*sum(harvest)
    model.min_80_constraint = Constraint(rule=min_80_constraint_rule)

    # * Constraint 8
    # Constraint for pellets flow balance
    def pellet_flow_balance_rule(model, j):
        return (
            sum(model.pellet_flow[j, k] for k in model.refineries) - sum(model.harvest_flow[i, j] for i in model.locations) <= 1e-03
        )
    model.pellet_flow_balance_constraint = Constraint(model.depots, rule=pellet_flow_balance_rule)

    def flow_binary_relation_rule(model, i, j):
        """
        This constraint ensures that if a depot is used (the binary variable is 1), 
        the corresponding harvest flow can be non-zero, but if a depot is not used (the binary variable is 0), the corresponding harvest flow must be zero.
        """
        return model.harvest_flow[i, j] <= model.depots_used[j] * harvest[i-1]
    model.flow_binary_relation_constraint = Constraint(model.locations, model.depots, rule=flow_binary_relation_rule)


    def pellet_flow_binary_relation_rule(model,i, j, k):
        return model.pellet_flow[j, k] <= model.refineries_used[k] * sum(model.harvest_flow[i, j] for i in model.locations)
    model.pellet_flow_binary_relation_constraint = Constraint(model.locations, model.depots, model.refineries, rule=pellet_flow_binary_relation_rule)


    # Solve the problem
    print("Solution started...")
    solver = SolverFactory('ipopt')
    results = solver.solve(model)

    # Display results
    print("Solver status:", results.solver.status)
    print("Objective value:", value(model.objective))

    depots_used = [j for j in model.depots if value(model.depots_used[j]) > 0.5]
    # print("Depot locations:", depots_used)
    refineries_used = [k for k in model.refineries if value(model.refineries_used[k]) > 0.5]
    # print("Refineries locations:", refineries_used)

    biomass_source_index = []
    biomass_destination_index = []
    biomass_flow = []
    for i in model.locations:
        for j in depots_used:
            f = value(model.harvest_flow[i, j])
            if f > 0:
                biomass_source_index.append(i)
                biomass_destination_index.append(j)
                biomass_flow.append(f)

    assert (all(x >= 0 for x in biomass_flow)),  "Constraint 1 failed for biomass demand supply"
    
    pellet_source_index = []
    pellet_destination_index = []
    pellet_flow = []
    for i in model.locations:
        for j in refineries_used:
            f = value(model.pellet_flow[i, j])
            if f > 0:
                pellet_source_index.append(i)
                pellet_destination_index.append(j)
                pellet_flow.append(f)
                
    assert (all(x >= 0 for x in pellet_flow)),  "Constraint 1 failed for pellet demand supply"

    return [depots_used, refineries_used, biomass_source_index, biomass_destination_index, biomass_flow, pellet_source_index, pellet_destination_index, pellet_flow]